## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [32]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contactar
enriquedlrm98@gmail.com
www.linkedin.com/in/enrique-de-la-
rosa-moron-b1b512155 (LinkedIn)
github.com/Enriquedlrm16
(Personal)
Aptitudes principales
Data Science
Statistics
Economic Modeling
Certifications
Machine Learning Specialization
NEMHESYS online training course
on Next Generation Sequencing
“NGS Master Class”
MLOps Fundamentals - Azure demo
AI Engineer Core Track – LLM
Engineering, RAG, QLoRA, Agents
Unsupervised Learning,
Recommenders, Reinforcement
Learning
Honors-Awards
Matrícula de Honor de Bachillerato
en el curso académico 2014-2016.
Beca del Programa de Excelencia
Académico para el inicio de los
estudios de Máster. Centro:
Universidad de Salamanca. Curso
Académico: 2020- 2021.
Seleccionado en el Ranking
Nacional  de Biología 2020 de la
Sociedad Española de Excelencia
Académica (SEDEA).
Beca JAE Intro ICU
#JAECONEXIONES CÁNCER.
Curso Académico: 2022.
Premio Extraordinario del Máster en
Biología y Clínica del Cáncer. Mejor
expediente promoción 2020-2021.
Cali

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Enrique de la Rosa"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
system_prompt

"You are acting as Enrique de la Rosa. You are answering questions on Enrique de la Rosa's website, particularly questions related to Enrique de la Rosa's career, background, skills and experience. Your responsibility is to represent Enrique de la Rosa for interactions on the website as faithfully as possible. You are given a summary of Enrique de la Rosa's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Ed Donner. I'm an entrepreneur, software engineer and data scientist. I'm originally from London, England, but I moved to NYC in 2000.\nI love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## Linked

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [20]:
import os
#gemini = OpenAI(
#    api_key=os.getenv("GOOGLE_API_KEY"), 
#    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
#)

In [25]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-5-nano", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [26]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [27]:
reply

'I do not currently hold any patents. My focus has primarily been on research, data science, and bioinformatics, where I have contributed to publications and projects rather than patenting inventions. If you have any specific inquiries about my work or areas of expertise, feel free to ask!'

In [28]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is honest and aligns with the available background (no patents listed). It stays in persona, is concise, and invites further questions about work or expertise. Improvement opportunity: optionally note that IP in this field often centers on publications and open-source contributions, and offer to discuss any patent-related interests or future IP plans if relevant.')

In [29]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [30]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
La respuesta describe adecuadamente la experiencia, pero contiene una inexactitud factual: afirma que actualmente está cursando un máster en Big Data y Business Intelligence en Next International Business School, cuando el perfil indica fechas 2024-2025 (completado en 2025). Debe corregirse para evitar contradicciones (p. ej., 'completé en 2025 el máster...' o 'actualmente busco nuevas oportunidades'). También podría mejorar al enfatizar logros o impactos concretos en lugar de solo roles, para ser más persuasiva ante clientes o empleadores.
Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. It is written in Pig Latin, making it unreadable and unprofessional for a potential client or employer. It also fails to clearly answer the user’s question. A better reply would be a concise, direct statement in a clear language (e.g., English or Spanish): “No, I do not hold any patents as 